# Webpage classification - Fetching data from HTML files

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from bs4 import BeautifulSoup
from string import punctuation
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
% matplotlib inline

In [2]:
STOPWORDS = stopwords.words('english')

## Fetch data

In [3]:
class fetch_data(object):
    """
    This class helps to fetch the data from
    the HTML files to a structured one.
    """
    
    def __init__(self, directory):
        """
        The constuctor of the class.
        
        Arguments:
        
        1. directory: The directory of the files.
        """
        
        self.data_frame = pd.DataFrame(columns = ["URL", "Text", "University", "Label"])
        # dataframe to load data into
        self.dir = directory
        # input files directory
        self.lemmatizer = WordNetLemmatizer()
        # lemmatizer object
        
    def __text_process(self, text):
        """
        This function helps to perform text processing
        by removing stop words, and lemmatizing the words.
        
        Arguments:
        
        1. self: The object.
        2. text: The input text to be processed.
        
        Return:
        1. text: Processed text.
        """
        
        text = text.replace("cs", "computer science")
        # replace cs with computer science
        text_lst = text.split(" ")
        # get list of words
        text_lst = [self.lemmatizer.lemmatize(word.decode("utf8")) for word in text_lst if word not in STOPWORDS and len(word) > 1]
        # process text
        return " ".join(text_lst)
        
    def __get_text(self, filename):
        """
        This function helps to get the text
        data, and the anchor text for a given
        HTML file name.
        
        Arguments:
        
        1. self: The object.
        2. filename: The filename from which the data
        is to be extracted.
        """
        
        with open(filename) as obj:
            # open file to read data
            data = obj.read()
            # read data from file
        data = re.sub('^[^<]+', "", data)
        # remove top headers from the file
        data_bs = BeautifulSoup(data)
        # beautiful soup processed data
        text = data_bs.get_text()
        # get the text data alone
        text = text.encode("utf8").translate(None, punctuation)
        # remove all punctuations from the data
        text = text.replace("\n", " ")
        # replace new line with space
        text = re.sub("\d", "", text)
        # remove all digits from the text
        text = re.sub("[\s]{2,}", " ", text).lower()
        # replace multiple space with single space
        text = self.__text_process(text)
        # func call to perform further text processing
        return text
        
            
    def __get_filename(self):
        """
        This function acts as a generator
        to yield filename with complete path
        which assists in further process.
        """
        
        index = 0
        # set index for the dataframe
        for path, _, file_lst in os.walk(self.dir):
            # iterate over sub-directories to the files
            for f in file_lst:
                # iterate over each file in the sub-directory
                self.data_frame.loc[index, "URL"] = f.strip("^")
                # add URL to the dataframe
                path_lst = path.split("\\")
                # get path traversed in list
                self.data_frame.loc[index, "University"] = path_lst[-1]
                # add University to the dataframe
                self.data_frame.loc[index, "Label"] = path_lst[-2]
                # add the category to the dataframe
                yield index, os.path.join(path, f)
                # yield the index, and the file path
                index += 1
                # update index
        
    def get_data(self):
        """
        This function traverses the sub-directories
        to each filename, and fetches the required data
        after pre-processing.
        """
        
        file_generator = self.__get_filename()
        # generator to get filename
        for index, file_path in file_generator:
            # iterate over index, and filepath
            text = self.__get_text(file_path)
            # get text, and anchor contents of the file
            self.data_frame.loc[index, "Text"] = text.encode("utf8")
            # add text data to the dataframe
            
    

### Fetching data 

In [4]:
crawl = fetch_data(directory = "webkb")
# initialize object to extract data

# NOTE: The argument directory is subjective to change

In [5]:
crawl.get_data()
# fetch the required data

C:\Users\c001384\AppData\Local\Continuum\anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\Users\c001384\AppData\Local\Continuum\anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
C:\Users\c001384\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:41: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [6]:
crawl.data_frame.to_csv(path_or_buf = "data.csv", index = False)
# write data to CSV file named "data.csv"